In [33]:
dependency_graph = {
    "x1" :[],   
    "x2" : ["x1"],
    "x3" : ["x1", "x2"],
    "x4" : ["x1", "x2", "x3"],
    "x5" : ["x1", "x2", "x3", "x4"],
}

In [34]:
import numpy as np
import itertools


class DistributionOld(object):  
    def __init__(self, num_variables, dependency_graph = None,  initial_parameters=None):
        self.num_variables = num_variables

        self.parameters    = initial_parameters if initial_parameters is not None else self.generate_parameters(num_variables)       
        self.learning_rate  = 0.001
    
    def generate_parameters(self,len_x):
        # k -> 2^(k-1)
        elements = [0,1]
        dict_parameters = { "x1": {() : np.random.uniform(low=0.0, high=1.0, size=None)}}
        for k in range(2, len_x+1):
            permutations = list(itertools.product(elements, repeat=k-1))
            dict_parameters["x"+str(k)] = {}
            for j in range(len(permutations)):
                 dict_parameters["x"+str(k)][permutations[j]] = np.random.uniform(low=0.0, high=1.0, size=None)
        return dict_parameters
    
    def forward_process(self, dict_parameters):
        elements = [0,1]
        dict_new_parameters = { "x1": {() : self.sigmoid(dict_parameters["x1"][()])   }}
        for k in range(2, self.num_variables+1):
            permutations = list(itertools.product(elements, repeat=k-1))
            dict_new_parameters["x"+str(k)] = {}
            for j in range(len(permutations)):
                    dict_new_parameters["x"+str(k)][permutations[j]] = self.sigmoid(dict_parameters["x"+str(k)][permutations[j]] )
        
        return dict_new_parameters

    def _generate_prob_sample(self,prob_dist):
        list_sample = []
        prob = []
        len_x = len(prob_dist.keys())
        for i in range(1, len_x+1):
            p = prob_dist["x"+str(i)][tuple(list_sample)]
            sample = np.random.binomial(n=1, p=p)
            if sample == 0:
                p = 1-p
                prob += [float(p)]
            else:
                prob += [float(p)]
            list_sample +=[int(sample)]
        return prob, list_sample
    
    def get_sample_prob(self, sample, prob_dist):
        prob = 1.0
        for i in range(1,len(sample)+1):
            p = prob_dist["x"+str(i)][tuple(sample[:i-1])]
            prob *= 1-p if sample[i-1] == 0 else  p
        return prob 
    
    def generate_samples(self,num_samples):
        list_samples = []
        prob_dist = self.forward_process(self.parameters)
        for i in range(num_samples):
            prob, sample = self._generate_prob_sample(prob_dist=prob_dist)
            prob_product = round(np.prod(prob),6)
            list_samples +=[(sample,prob_product)]
        return list_samples

    def population_probility(self,total_sample):
        N = len(total_sample)
        unique_set = [list(x) for x in set(tuple(x) for x in total_sample)]
        prob_dict = {}
        for i in range(len(unique_set)):
            prob_dict[tuple(unique_set[i])] = total_sample.count(unique_set[i])/N 
        return prob_dict
    # kl divergence = ∑ p(x) log p(x) - p(x) log q(x)
    def kl_divergence(self,  sample_X, other ):
        kl_sum = 0.0
        self_prob_dist = self.forward_process(self.parameters)
        other_prob_dist = other.forward_process(other.parameters)
        for sample in sample_X:
            self_prob  = self.get_sample_prob(sample,prob_dist=self_prob_dist)
            other_prob = other.get_sample_prob(sample,prob_dist=other_prob_dist )
            kl_sum += self_prob*np.log(self_prob) - self_prob * np.log(other_prob)
        return kl_sum
    
    def sigmoid(self,input):
        return  1 / (1 + np.exp(-input))

    def sigmoid_derivative(self, sig):
        return  sig * (1-sig)
    

    # log_likehood = ∑log p(x)
    def log_likelihood(self,sample_X):
        log_likelihood_sum = 0.0
        prob_dist = self.forward_process(self.parameters)
        for sample in sample_X:
            self_prob = self.get_sample_prob(sample=sample, prob_dist=prob_dist)
            log_likelihood_sum += np.log(self_prob+  1e-10)
        
        return log_likelihood_sum

    def entropy(self,sample_X):
        entropy_sum = 0.0
        for sample in sample_X:
            self_prob = self.get_sample_prob(sample=sample)
            entropy_sum += self_prob * np.log(self_prob)
        
        return entropy_sum
    
    def cross_entropy(self, sample_X, other):
        cross_entropy_sum = 0.0
        for sample in sample_X:
            self_prob = self.get_sample_prob(sample=sample)
            other_prob = other.get_sample_prob(sample=sample)
            cross_entropy_sum += self_prob * np.log(other_prob)
        return cross_entropy_sum
    
    # def _initialize_derivates(self):
    #     # k -> 2^(k-1)
    #     elements = [0,1]
    #     dict_parameters = { "dx1": {() : 0.0}}
    #     for k in range(2, self.num_variables+1):
    #         permutations = list(itertools.product(elements, repeat=k-1))
    #         dict_parameters["dx"+str(k)] = {}
    #         for j in range(len(permutations)):
    #              dict_parameters["dx"+str(k)][permutations[j]] = 0.0 
    #     return dict_parameters
 

    def _initialize_derivatives(self):
        elements = [0, 1]
        dict_derivs = {"dx1": {(): 0.0}}
        for k in range(2, self.num_variables + 1):
            permutations = list(itertools.product(elements, repeat=k - 1))
            dict_derivs["dx" + str(k)] = {perm: 0.0 for perm in permutations}
        return dict_derivs

    # def logp_derivate(self, sample_X):
    #     derviate_sums = self._initialize_derivates()
    #     prob_dist = self.farword_process(dict_parameters=self.parameters)
    #     for sample in sample_X:
    #         #[1,0,0,1,1]
    #         p_comp = prob_dist["x1"][()]
    #         derviate_sums["dx1"][()] +=   self.derviate_sigmoid(p_comp)/p_comp 
    #         for i in range(2,len(sample)+1):
    #             cond_variable = tuple(sample[:i-1])
    #             p_comp = prob_dist["x"+str(i)][cond_variable]
    #             derviate_sums["dx"+str(i)][cond_variable] += self.derviate_sigmoid(p_comp)/p_comp
        
    #     derviate_sums["dx1"][()] = derviate_sums["dx1"][()] / len(sample_X)
    #     for i in range(2,len(sample)+1):
    #         cond_variable = tuple(sample[:i-1])
    #         derviate_sums["dx"+str(i)][cond_variable] = derviate_sums["dx"+str(i)][cond_variable]/ len(sample_X)

    #     return derviate_sums


    def logp_derivative(self, sample_X):
        deriv_sums = self._initialize_derivatives()
        prob_dist = self.forward_process(self.parameters)
        
        for sample in sample_X:
            # For each sample, calculate the gradient of log likelihood
            for i in range(1, len(sample) + 1):
                cond_var = tuple(sample[:i - 1])
                p = prob_dist["x" + str(i)][cond_var]
                
                # Adjust gradient based on sample outcome (1 or 0)
                grad = self.sigmoid_derivative(p) * (sample[i - 1] - p) / (p * (1 - p) + 1e-10)
                deriv_sums["dx" + str(i)][cond_var] += grad
        
        # Average gradients over all samples
        for k in deriv_sums:
            for key in deriv_sums[k]:
                deriv_sums[k][key] /= len(sample_X)
        
        return deriv_sums 
    
    def update_parameters(self, derviate_sums):
        self.parameters["x1"][()] = self.parameters["x1"][()] + self.learning_rate * derviate_sums["dx1"][()] 
        for i in range(2,self.num_variables+1):
            for tup in self.parameters["x"+str(i)]:
                self.parameters["x"+str(i)][tup] = self.parameters["x"+str(i)][tup] + self.learning_rate *  derviate_sums["dx"+str(i)][tup]
   


In [38]:
import numpy as np
import itertools

class Distribution(object):
    def __init__(self, num_variables, initial_parameters=None):
        self.num_variables = num_variables
        self.parameters = initial_parameters if initial_parameters is not None else self.generate_parameters(num_variables)
        self.learning_rate = 0.001

    def generate_parameters(self, len_x):
        elements = [0, 1]
        dict_parameters = {"x1": {(): np.random.uniform(0.0, 1.0)}}
        for k in range(2, len_x + 1):
            permutations = list(itertools.product(elements, repeat=k - 1))
            dict_parameters["x" + str(k)] = {perm: np.random.uniform(0.0, 1.0) for perm in permutations}
        return dict_parameters

    def forward_process(self, dict_parameters):
        elements = [0, 1]
        dict_new_parameters = {"x1": {(): self.sigmoid(dict_parameters["x1"][()])}}
        for k in range(2, self.num_variables + 1):
            permutations = list(itertools.product(elements, repeat=k - 1))
            dict_new_parameters["x" + str(k)] = {perm: self.sigmoid(dict_parameters["x" + str(k)][perm]) for perm in permutations}
        return dict_new_parameters

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, sig):
        return sig * (1 - sig)

    def _generate_prob_sample(self, prob_dist):
        list_sample, prob = [], []
        for i in range(1, len(prob_dist) + 1):
            p = prob_dist["x" + str(i)][tuple(list_sample)]
            sample = np.random.binomial(1, p)
            prob.append(p if sample == 1 else 1 - p)
            list_sample.append(sample)
        return prob, list_sample

    def logp_derivative(self, sample_X):
        deriv_sums = self._initialize_derivatives()
        prob_dist = self.forward_process(self.parameters)
        
        for sample in sample_X:
            # For each sample, calculate the gradient of log likelihood
            for i in range(1, len(sample) + 1):
                cond_var = tuple(sample[:i - 1])
                p = prob_dist["x" + str(i)][cond_var]
                

                # Adjust gradient based on sample outcome (1 or 0)
                #grad =   p * (1 - p) * (sample[i - 1] - p) / (p * (1 - p) + 1e-10)

                grad = (1-p) if sample[i-1]==1 else -p 
                #self.sigmoid_derivative(p) * (sample[i - 1] - p) / (p * (1 - p) + 1e-10)
                deriv_sums["dx" + str(i)][cond_var] += grad
        
        # Average gradients over all samples
        for k in deriv_sums:
            for key in deriv_sums[k]:
                deriv_sums[k][key] /= len(sample_X)
        
        return deriv_sums

    def _initialize_derivatives(self):
        elements = [0, 1]
        dict_derivs = {"dx1": {(): 0.0}}
        for k in range(2, self.num_variables + 1):
            permutations = list(itertools.product(elements, repeat=k - 1))
            dict_derivs["dx" + str(k)] = {perm: 0.0 for perm in permutations}
        return dict_derivs

    def generate_samples(self, num_samples):
        list_samples = []
        prob_dist = self.forward_process(self.parameters)  # Use updated parameter probabilities
        for _ in range(num_samples):
            prob, sample = self._generate_prob_sample(prob_dist=prob_dist)
            prob_product = round(np.prod(prob), 6)
            list_samples.append((sample, prob_product))  # Store both sample and its probability
        return list_samples
    def update_parameters(self, deriv_sums):
        self.parameters["x1"][()] += self.learning_rate * deriv_sums["dx1"][()]
        for i in range(2, self.num_variables + 1):
            for perm in self.parameters["x" + str(i)]:
                self.parameters["x" + str(i)][perm] += self.learning_rate * deriv_sums["dx" + str(i)][perm]

    def log_likelihood(self, sample_X):
        log_likelihood_sum = 0.0
        prob_dist = self.forward_process(self.parameters)
        for sample in sample_X:
            sample_prob = self.get_sample_prob(sample, prob_dist)
            log_likelihood_sum += np.log(sample_prob + 1e-10)
        return log_likelihood_sum

    def get_sample_prob(self, sample, prob_dist):
        prob = 1.0
        for i in range(1, len(sample) + 1):
            p = prob_dist["x" + str(i)][tuple(sample[:i - 1])]
            prob *= p if sample[i - 1] == 1 else (1 - p)
        return prob

    def kl_divergence(self, sample_X, other):
        kl_sum = 0.0
        epsilon = 1e-10  # Small constant to avoid division by zero
        self_prob_dist = self.forward_process(self.parameters)
        other_prob_dist = other.forward_process(other.parameters)
        
        for sample in sample_X:
            self_prob = self.get_sample_prob(sample, prob_dist=self_prob_dist)
            other_prob = other.get_sample_prob(sample, prob_dist=other_prob_dist)
            
            # Add epsilon to both probabilities to handle log(0) cases
            kl_sum += self_prob * np.log((self_prob + epsilon) / (other_prob + epsilon))
        
        return kl_sum


In [39]:
# # Original Distribution
# num_variables  = 5
# initial_parameters = {'x1': {(): np.float64(0.6105838933937734)}, 'x2': {(0,): np.float64(1.7219534503097305), (1,): np.float64(2.120384662660463)}, 'x3': {(0, 0): np.float64(1.205721147176752), (0, 1): np.float64(1.7663034765656154), (1, 0): np.float64(1.5868785797181886), (1, 1): np.float64(2.2475606318724775)}, 'x4': {(0, 0, 0): np.float64(0.8718638681461294), (0, 0, 1): np.float64(1.3777744788928312), (0, 1, 0): np.float64(1.3872575613238112), (0, 1, 1): np.float64(1.6344756295373246), (1, 0, 0): np.float64(1.2283292311493113), (1, 0, 1): np.float64(1.3844796344236854), (1, 1, 0): np.float64(1.9171894239876563), (1, 1, 1): np.float64(1.8072898440282397)}, 'x5': {(0, 0, 0, 0): np.float64(0.6594719945861921), (0, 0, 0, 1): np.float64(0.8885058162122675), (0, 0, 1, 0): np.float64(0.6757744234817903), (0, 0, 1, 1): np.float64(1.0225089642190064), (0, 1, 0, 0): np.float64(0.6843202607735592), (0, 1, 0, 1): np.float64(0.8031412634690331), (0, 1, 1, 0): np.float64(1.1444794752930405), (0, 1, 1, 1): np.float64(1.1141948522569136), (1, 0, 0, 0): np.float64(1.083513235037557), (1, 0, 0, 1): np.float64(0.6979658130989729), (1, 0, 1, 0): np.float64(0.9058050137043484), (1, 0, 1, 1): np.float64(1.1277455484666434), (1, 1, 0, 0): np.float64(1.13679609761262), (1, 1, 0, 1): np.float64(1.6394609678500875), (1, 1, 1, 0): np.float64(1.288949071159566), (1, 1, 1, 1): np.float64(1.4528234487164917)}}
# ori_dist_obj   = Distribution(num_variables=num_variables, initial_parameters=initial_parameters)
# data_X         = ori_dist_obj.generate_samples(num_samples=100000)
# sample_X       = [ temp[0] for temp in data_X]




In [40]:
# Original Distribution
num_variables  = 1
initial_parameters = {'x1': {(): np.float64(0.6105838933937734)}}
ori_dist_obj   = Distribution(num_variables=num_variables, initial_parameters=initial_parameters)
data_X         = ori_dist_obj.generate_samples(num_samples=100000)
sample_X       = [ temp[0] for temp in data_X]




In [41]:
####### Initialize a random distribution 
learn_dist_obj = Distribution(num_variables=num_variables)



In [42]:
batch_size = 1000 
num_batches = len(sample_X)//batch_size

In [43]:
# Original Distribution
num_variables = 5

initial_parameters = {'x1': {(): np.float64(0.6105838933937734)}, 
                      'x2': {(0,): np.float64(1.7219534503097305), (1,): np.float64(2.120384662660463)}, 
                      'x3': {(0, 0): np.float64(1.205721147176752), (0, 1): np.float64(1.7663034765656154), (1, 0): np.float64(1.5868785797181886), (1, 1): np.float64(2.2475606318724775)},
                      'x4': {(0, 0, 0): np.float64(0.8718638681461294), (0, 0, 1): np.float64(1.3777744788928312), (0, 1, 0): np.float64(1.3872575613238112), (0, 1, 1): np.float64(1.6344756295373246), (1, 0, 0): np.float64(1.2283292311493113), (1, 0, 1): np.float64(1.3844796344236854), (1, 1, 0): np.float64(1.9171894239876563), (1, 1, 1): np.float64(1.8072898440282397)},
                      'x5': {(0, 0, 0, 0): np.float64(0.6594719945861921), (0, 0, 0, 1): np.float64(0.8885058162122675), (0, 0, 1, 0): np.float64(0.6757744234817903), (0, 0, 1, 1): np.float64(1.0225089642190064), (0, 1, 0, 0): np.float64(0.6843202607735592), (0, 1, 0, 1): np.float64(0.8031412634690331), (0, 1, 1, 0): np.float64(1.1444794752930405), (0, 1, 1, 1): np.float64(1.1141948522569136), (1, 0, 0, 0): np.float64(1.083513235037557), (1, 0, 0, 1): np.float64(0.6979658130989729), (1, 0, 1, 0): np.float64(0.9058050137043484), (1, 0, 1, 1): np.float64(1.1277455484666434), (1, 1, 0, 0): np.float64(1.13679609761262), (1, 1, 0, 1): np.float64(1.6394609678500875), (1, 1, 1, 0):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    np.float64(1.288949071159566), (1, 1, 1, 1): np.float64(1.4528234487164917)}}
#initial_parameters = {'x1': {(): np.float64(0.9)}, 'x2': {(0,): np.float64(0.6), (1,): np.float64(1.2)}} 
ori_dist_obj = Distribution(num_variables=num_variables, initial_parameters=initial_parameters)
data_X = ori_dist_obj.generate_samples(num_samples=100000)
sample_X = [temp[0] for temp in data_X]

# Initialize a random distribution
learn_dist_obj = Distribution(num_variables=num_variables)

# Training loop with increased learning rate and gradient averaging
learn_dist_obj.learning_rate = 0.01  # Try a larger learning rate

batch_size = 1000
num_batches = len(sample_X) // batch_size
num_epochs = 500

for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch {epoch}/{num_epochs}")
    
    for i in range(num_batches):
        sample_data = sample_X[i * batch_size: batch_size * (i + 1)]
        # Calculate and update gradients
        deriv_sums = learn_dist_obj.logp_derivative(sample_X=sample_data)
        learn_dist_obj.update_parameters(deriv_sums=deriv_sums)
        
        
    # Monitor learning progress each epoch
    curr_kl_div = ori_dist_obj.kl_divergence(sample_X=sample_X, other=learn_dist_obj)
    log_likelihood = learn_dist_obj.log_likelihood(sample_X=sample_X)
    print(learn_dist_obj.parameters)
    print(f"KL Divergence: {curr_kl_div:.6f} | Log Likelihood: {log_likelihood:.6f}")



Epoch 1/500
{'x1': {(): np.float64(0.6961722487230981)}, 'x2': {(0,): np.float64(0.574974275134936), (1,): np.float64(0.37746468247332393)}, 'x3': {(0, 0): np.float64(0.1509225942332232), (0, 1): np.float64(0.9087058093621988), (1, 0): np.float64(0.28250668586426997), (1, 1): np.float64(0.8292077709883985)}, 'x4': {(0, 0, 0): np.float64(0.6742730761913793), (0, 0, 1): np.float64(0.30434600554600166), (0, 1, 0): np.float64(0.5236076673817133), (0, 1, 1): np.float64(0.7921143306500226), (1, 0, 0): np.float64(0.5873801825217183), (1, 0, 1): np.float64(0.5646103261723401), (1, 1, 0): np.float64(0.24798389012864214), (1, 1, 1): np.float64(0.5605455758022515)}, 'x5': {(0, 0, 0, 0): np.float64(0.279994841684327), (0, 0, 0, 1): np.float64(0.881529848952757), (0, 0, 1, 0): np.float64(0.2103428370981739), (0, 0, 1, 1): np.float64(0.7505094004064583), (0, 1, 0, 0): np.float64(0.48275835595419436), (0, 1, 0, 1): np.float64(0.6376129625776819), (0, 1, 1, 0): np.float64(0.06409989998885506), (0, 1,

In [23]:

initial_parameters = {'x1': {(): np.float64(0.6105838933937734)}, 
                      'x2': {(0,): np.float64(1.7219534503097305), (1,): np.float64(2.120384662660463)}, 
                      'x3': {(0, 0): np.float64(1.205721147176752), (0, 1): np.float64(1.7663034765656154), (1, 0): np.float64(1.5868785797181886), (1, 1): np.float64(2.2475606318724775)},
                      'x4': {(0, 0, 0): np.float64(0.8718638681461294), (0, 0, 1): np.float64(1.3777744788928312), (0, 1, 0): np.float64(1.3872575613238112), (0, 1, 1): np.float64(1.6344756295373246), (1, 0, 0): np.float64(1.2283292311493113), (1, 0, 1): np.float64(1.3844796344236854), (1, 1, 0): np.float64(1.9171894239876563), (1, 1, 1): np.float64(1.8072898440282397)},
                      'x5': {(0, 0, 0, 0): np.float64(0.6594719945861921), (0, 0, 0, 1): np.float64(0.8885058162122675), (0, 0, 1, 0): np.float64(0.6757744234817903), (0, 0, 1, 1): np.float64(1.0225089642190064), (0, 1, 0, 0): np.float64(0.6843202607735592), (0, 1, 0, 1): np.float64(0.8031412634690331), (0, 1, 1, 0): np.float64(1.1444794752930405), (0, 1, 1, 1): np.float64(1.1141948522569136), (1, 0, 0, 0): np.float64(1.083513235037557), (1, 0, 0, 1): np.float64(0.6979658130989729), (1, 0, 1, 0): np.float64(0.9058050137043484), (1, 0, 1, 1): np.float64(1.1277455484666434), (1, 1, 0, 0): np.float64(1.13679609761262), (1, 1, 0, 1): np.float64(1.6394609678500875), (1, 1, 1, 0):
          

SyntaxError: incomplete input (539806723.py, line 6)

In [ ]:
for batch in [sample_X]:
    kl_dive = ori_dist_obj.kl_divergence(sample_X=batch, other=ori_dist_obj)
    ori_loglike = ori_dist_obj.log_likelihood(sample_X=batch)
    ori_entropy = ori_dist_obj.entropy(sample_X=batch)

    other_loglike = learn_dist_obj.log_likelihood(sample_X=batch)
    other_entropy = learn_dist_obj.entropy(sample_X=batch)

    ori_other_entropy = ori_dist_obj.cross_entropy(sample_X=batch, other=learn_dist_obj)

    print(f"kl : {kl_dive}, ori-loglike : {ori_loglike}, ori-entropy : {ori_entropy}") 
    print(f"other-loglike : {other_loglike}, other-entropy : {other_entropy}, ori-other-entropy : {ori_other_entropy}")


In [ ]:
ori_dist_obj.entropy(sample_X=sample_X)

In [25]:
dist_obj1         = Distribution(num_variables=num_variables)


In [26]:
dist_obj2         = Distribution(num_variables=num_variables)


In [ ]:
dist_obj.kl_divergence(p_data=sample_p, sample_X=sample_X, other=dist_obj2)